<a href="https://colab.research.google.com/github/divyamdholwani/Facial-Emotions-Recognition/blob/main/MAIOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [4]:
import csv
data ={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [5]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [6]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [7]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [8]:
from shutil import copyfile
split_size=0.8

for emotion, images in data.items():
    train_size= int(split_size*len(images)) 
    train_images = images[:train_size] 
    test_images = images[train_size:] 
    
    for image in train_images: 
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/training',emotion, image)
        copyfile(source, dest)
    
    for image in test_images: 
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/testing',emotion, image)
        copyfile(source, dest)
  

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [10]:
model =tf.keras.models.Sequential([
    Conv2D(16,(3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(8, activation='softmax')

])

model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [12]:
train_dir= "‪/content/master_data/training"

test_dir= '‪/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
train_dir.strip("‪u202c"), target_size=(100,100),class_mode='categorical', batch_size=128)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
test_dir.strip("‪u202c"), target_size=(100,100),class_mode='categorical', batch_size=128)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [13]:
es = EarlyStopping(monitor='val_acc', patience=2, min_delta=0.01)

In [14]:
model.fit_generator(train_generator,
                   epochs=10,
                   verbose=1,
                   validation_data= test_generator,
                   callbacks=[es])

<ipython-input-14-d761873881a5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 29s 219ms/step - loss: 0.9502 - acc: 0.6123 - val_loss: 1.4409 - val_acc: 0.6225
Epoch 2/10
86/86 [==============================] - 19s 225ms/step - loss: 0.6235 - acc: 0.7930 - val_loss: 1.2045 - val_acc: 0.6696
Epoch 3/10
86/86 [==============================] - 18s 210ms/step - loss: 0.5239 - acc: 0.8214 - val_loss: 1.5763 - val_acc: 0.6437
Epoch 4/10
86/86 [==============================] - 18s 208ms/step - loss: 0.4768 - acc: 0.8384 - val_loss: 1.4209 - val_acc: 0.6845
Epoch 5/10
86/86 [==============================] - 20s 230ms/step - loss: 0.4329 - acc: 0.8472 - val_loss: 1.5096 - val_acc: 0.6922
Epoch 6/10
86/86 [==============================] - 18s 210ms/step - loss: 0.4040 - acc: 0.8621 - val_loss: 1.9949 - val_acc: 0.6856


PREDICTION!

In [42]:
filenames = test_generator.filenames
nb_samples = len(filenames)
filenames[10]


'anger/Dwarakish_159.jpg'

In [25]:
predict = model.predict_generator(test_generator,nb_samples)

<ipython-input-25-591d7ccab351>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = model.predict_generator(test_generator,nb_samples)


In [28]:
predict[0]

array([3.9693573e-04, 1.5780717e-05, 6.9928647e-05, 3.2293490e-05,
       9.9484247e-01, 4.1279513e-03, 4.2961023e-04, 8.5098327e-05],
      dtype=float32)

In [35]:
label_map = (train_generator.class_indices)
label_map = dict([(value, key) for key, value in label_map.items()])
label_map

{0: 'anger',
 1: 'contempt',
 2: 'disgust',
 3: 'fear',
 4: 'happiness',
 5: 'neutral',
 6: 'sadness',
 7: 'surprise'}

In [ ]:
import numpy as np
label_map[np.argmax(predict[10])]


In [44]:
automl = pickle.load(open('model_pickle','rb'))

**USING PICKLE MODEL TO IMPLEMENT!!**





In [15]:
import pickle
with open('model_pickle','wb') as f:
    pickle.dump(model,f)

In [45]:
predict = automl.predict_generator(test_generator,nb_samples)

<ipython-input-45-ee39da4308f8>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = automl.predict_generator(test_generator,nb_samples)


In [47]:
label_map[np.argmax(predict[1])]

'disgust'